# Extracte

In [ ]:
import pandas as pd
import requests
from sqlalchemy import create_engine, text


In [ ]:
def extract_csv(list_csv):


    api_url = "https://www.data.gouv.fr/api/2/datasets/5b598be088ee387c0c353714/resources/?page=1&page_size=50" # La requette est sur le /states/all et c'est la que les param sont important et préciser
    

    
    try:
        print("Making API request... (this may take a few seconds)")
        


        response = requests.get(api_url, timeout=10)
        


        if response.status_code == 200: #status_code = le statu html de la reponse et 200 c'est quand c'est bon
            print("Cela fonctionne")



            data = response.json() # Vas cherche les données de la request (voir ca comme les ROS)
        
            for part in data.get('data', []):

                title = part.get('title')
                

                if title in  list_csv:
                    
                    url_csv  = part.get('url')

                    requests_csv = requests.get(url_csv)

                    if requests_csv.status_code == 200:

                        with open("data/"+ title, "wb") as f:
                            f.write(requests_csv.content)


        print(f"Created all CSV")
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error fetching data: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error processing data: {e}")
        return pd.DataFrame()

In [ ]:
def extract_data(list_chemin):

    # print("📄 Reading tourism data from CSV...")
    
    try:

        df = pd.DataFrame()
        for chemin in list_chemin:
            df = pd.concat([df, pd.read_csv("../data/"+ chemin)], ignore_index=True)

        print('Data retreived in a dataframe')
        return df
        
    except Exception as e:
        print(f"❌ Error reading data: {e}")
        return pd.DataFrame()



In [ ]:
list_df = ["datatourisme-reg-ara.csv", "datatourisme-reg-bfc.csv", "datatourisme-reg-bre.csv",
        "datatourisme-reg-cor.csv", "datatourisme-reg-cvl.csv", "datatourisme-reg-gde.csv",
        "datatourisme-reg-hdf.csv", "datatourisme-reg-naq.csv", "datatourisme-reg-nor.csv",
        "datatourisme-reg-idf.csv",  "datatourisme-reg-occ.csv", "datatourisme-reg-pac.csv",
        "datatourisme-reg-pdl.csv"] # "datatourisme-reg-guf", "datatourisme-reg-myt", , "datatourisme-reg-reu", "datatourisme-reg-glp","datatourisme-reg-mtq",

In [ ]:
extract_csv(list_df)

df = extract_data(list_df)

# Transforme

In [ ]:
def clean_data(df_tourism):

    if df_tourism.empty:
        print("⚠️  No airport data to clean")
        return df_tourism
    
    print(f"🧹 Cleaning  data...")
    print(f"Starting with {len(df_tourism)} ")
    
    # Make a copy to avoid modifying the original
    df = df_tourism.copy()

    # Drop the usless data
    df.drop(['Periodes_regroupees', 'Covid19_mesures_specifiques', 'Contacts_du_POI', 'Classements_du_POI'], axis=1)


    # On récupère uniquement les catégorie de POI
    df["Categories_de_POI"] = df["Categories_de_POI"].str.split('/').str[-1]
    df["Categories_de_POI"] = df["Categories_de_POI"].str.split('#').str[-1]

    # On découpe la colone en 3 pour plus de clareter
    col_index = df.columns.get_loc("Code_postal_et_commune")

    df_temp = df["Code_postal_et_commune"].str.split('#', expand=True)
    df_temp.columns = ["Code_postale", "Commune"]
    df_temp["Département"] = df_temp["Code_postale"].str[0:2]

    df.drop(columns=["Code_postal_et_commune"], inplace=True)


    df.insert(col_index, "Département", df_temp["Département"])
    df.insert(col_index + 1, "Code_postale", df_temp["Code_postale"])
    df.insert(col_index + 2, "Commune", df_temp["Commune"])

    # On drop les valeur null
    df = df.dropna(subset=["Nom_du_POI", "Categories_de_POI", 'Latitude', 'Longitude']) # ,"Adresse_postale"]) 

    # Passage des dates en format date
    df["Date_de_mise_a_jour"] = pd.to_datetime(df["Date_de_mise_a_jour"])

    
    print(f'Il reste {len(df)} data après nettoyage.')  

    return df

# Load

In [ ]:
DATABASE_CONFIG = {
    'username': 'alexm',
    'password': '', 
    'host': 'localhost',
    'port': '5432',
    'database': 'airlife_db'
}

In [ ]:
def get_connection_string():
    """Build PostgreSQL connection string"""
    return f"postgresql://{DATABASE_CONFIG['username']}:{DATABASE_CONFIG['password']}@{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"

In [ ]:
def load_to_database(Tourisme_df):

    print("💾 Loading data to PostgreSQL database...")

    connection_string = get_connection_string()

    try:

        engine = create_engine(connection_string)

        Tourisme_df.to_sql("airports", con = engine, if_exists='replace', index=False)


        print(f"✅ Loaded {len(Tourisme_df)} airports to database")

    except Exception as e:
        print(f"❌ Error loading data to database: {e}")
        print("💡 Make sure:")
        print("   - PostgreSQL is running")
        print("   - Database 'airlife_db' exists") 
        print("   - Username and password are correct")
        print("   - Tables are created (run database_setup.sql)")